In [ ]:
# 시계열 데이터 :10일 데이터로 학습, 11일차 데이터 레이블
import numpy as np

data=[10,11,12,13,14,15,16]
# data                     label 
# [10,11,12,13,14]            15
# [11,12,13,14,15]            16
def create_seq(data, seq_length):
    seqs=[]
    target=[]
    # seqs → 입력 데이터 시퀀스들을 모은 리스트
# target → 각 시퀀스 다음에 나오는 값을 모은 리스트
    for i in range(len(data)-seq_length):
# data[i:i+seq_length] : 데이터에서 길이 seq_length 만큼 잘라서 입력 시퀀스를 만든다.
        seqs.append(data[i:i+seq_length])
        target.append(data[i+seq_length])
# data[i+seq_length] : 잘라낸 시퀀스 다음 값을 레이블(정답)로 만든다.
    return np.array(seqs), np.array(target)

seq_length=2

nd_data,target=create_seq(data, seq_length) #seq_length는 입력 시퀀스의 길이,
# seq_length 값을 크게 하면 과거를 더 길게 기억하고 예측 
#  주가 예측, 날씨, 매출 예측 같은데서 자주 쓰이는 기본 전처리 
# 몇 개의 연속된 데이터를 보고 다음 값을 예측할지를 결정
print(nd_data)
print(target)
        

[[10 11]
 [11 12]
 [12 13]
 [13 14]
 [14 15]]
[12 13 14 15 16]


In [5]:
# 임포트
import yfinance as yf
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [20]:
# 종목 정의 기간 설정 
tickers=['AAPL','MSFT','GOOGL']
start_date='2020-04-17'
end_date='2025-04-16'

# 데이터 로딩 
data=yf.download(
    tickers=tickers,
    start=start_date,
    end=end_date
)['Close']
data.head()

[*********************100%***********************]  3 of 3 completed


Ticker,AAPL,GOOGL,MSFT
Date,,,
2020-04-17,68.629608,63.645714,170.952148
2020-04-20,67.205093,62.757458,167.563705
2020-04-21,65.127777,60.319614,160.633743
2020-04-22,67.003677,62.621113,166.089676
2020-04-23,66.743996,63.256073,164.079575


In [10]:
data['Close'].head()
data['Close'].tail()

Ticker,AAPL,GOOGL,MSFT
Date,,,
2025-04-09,198.850006,158.710007,390.489990
2025-04-10,190.419998,152.820007,381.350006
2025-04-11,198.149994,157.139999,388.450012
2025-04-14,202.520004,159.070007,387.809998
2025-04-15,202.139999,156.309998,385.730011


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1256 entries, 2020-04-17 to 2025-04-15
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    1256 non-null   float64
 1   GOOGL   1256 non-null   float64
 2   MSFT    1256 non-null   float64
dtypes: float64(3)
memory usage: 39.2 KB


In [12]:
# 결측치 처리(보간법 이용-> 채우겠다)
data.interpolate(method='linear', inplace=True)


In [13]:
# 신경망은 중간에 결측치가 있으면 에러남 -> nan 처리 해야함 
# 스케일링 처리 : 각 종목별 가격 차이 심하다 
data['Close'].tail()

Ticker,AAPL,GOOGL,MSFT
Date,,,
2025-04-09,198.850006,158.710007,390.489990
2025-04-10,190.419998,152.820007,381.350006
2025-04-11,198.149994,157.139999,388.450012
2025-04-14,202.520004,159.070007,387.809998
2025-04-15,202.139999,156.309998,385.730011


In [22]:
# 스케일링 처리 : 각 종목별 가격 차이 심하다 
scaler=StandardScaler()
scaled_data=scaler.fit_transform(data)
scaled_data[-5:] #뒤에서 5개 가져오기

array([[0.93545652, 1.04211358, 1.11109945],
       [0.72634903, 0.86063224, 0.99553718],
       [0.91809263, 0.99373887, 1.08530679],
       [1.02649133, 1.05320584, 1.0772147 ],
       [1.01706525, 0.96816504, 1.0509162 ]])

In [17]:
# 시계열 데이터에서 데이터와 레이블 생성 
def create_seq(data, seq_length):
    seqs=[]
    targets=[]
    
    for i in range(len(data)-seq_length):
        seqs.append(data[i:i+seq_length])
        targets.append(data[i+seq_length])
    return np.array(seqs), np.array(targets)
     

In [16]:
1256-10

1246

In [ ]:
seq_length=10 #10일치 

X,y=create_seq(scaled_data, seq_length)
print(X[:2]) # 행렬 
print(y[:2])

[[[-2.2946781  -1.88698617 -1.66464828]
  [-2.33001338 -1.91435491 -1.70749039]
  [-2.38154149 -1.9894692  -1.79511003]
  [-2.33500953 -1.91855594 -1.72612742]
  [-2.34145097 -1.89899169 -1.75154233]
  [-2.29365408 -1.89066603 -1.71366267]
  [-2.29245008 -1.89946701 -1.7197134 ]
  [-2.32008069 -1.95814472 -1.77102708]
  [-2.26500039 -1.79011497 -1.67880828]
  [-2.22846149 -1.78318472 -1.65726636]]

 [[-2.33001338 -1.91435491 -1.70749039]
  [-2.38154149 -1.9894692  -1.79511003]
  [-2.33500953 -1.91855594 -1.72612742]
  [-2.34145097 -1.89899169 -1.75154233]
  [-2.29365408 -1.89066603 -1.71366267]
  [-2.29245008 -1.89946701 -1.7197134 ]
  [-2.32008069 -1.95814472 -1.77102708]
  [-2.26500039 -1.79011497 -1.67880828]
  [-2.22846149 -1.78318472 -1.65726636]
  [-2.25693407 -1.82823194 -1.71342016]]]
[[-2.25693407 -1.82823194 -1.71342016]
 [-2.23231364 -1.81967638 -1.66174417]]


In [26]:
# 데이터셋 : data + label  <= tensor 
# 텐서 변환 
# X_tensor=tprch.tensor
X_tensor=torch.tensor(X, dtype=torch.float32)
# X_tensor=torch.tensor(X, dtype=torch.float32) # numpy 와 list 둘다 가능
y_tensor=torch.tensor(y, dtype=torch.float32)

# dataset
dataset=TensorDataset(X_tensor, y_tensor)

# DataLoader : batch_size=32, shuffle=False
batch_size=32
dataLoader=DataLoader(dataset, batch_size=batch_size, shuffle=False)

# 데이터로더 
for xb, yb in dataLoader:
    print(f'입력 데이터 배치 크기 : {xb.shape}')
    print(f'레이블 배치 크기 : {yb.shape}')
    break


입력 데이터 배치 크기 : torch.Size([32, 10, 3])
레이블 배치 크기 : torch.Size([32, 3])


In [29]:
# 신경망 구성 
import torch.nn as nn 
import torch.optim as optim

class LSTMStockModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size): # 층구성
        super(LSTMStockModel,self).__init__()
        self.lstm=nn.LSTM(input_size, hidden_size, batch_first=True)
        self.layer1=nn.Linear(hidden_size, output_size)
    
    def forward(self, X):
        lstm_out,_=self.lstm(X)
        out=self.layer1(lstm_out[:,-1,:])
        return out #신경망을 거쳐서 나온 예측값 

In [ ]:
# 설정 
# 는 PyTorch 기반 LSTM 딥러닝 모델을 사용해 주가 예측을 학습하는 루프
# 여러 종목 주가 데이터로 lstm 모델로 학습해 미래의 주가를 예측하는 모델을 학습 
input_size=len(tickers) #입열되는 피처 수 (세종목) 3
hidden_size=32 # hidden_size 가 unit 의 개수가 아니다 
# LSTM : 셀
output_size=len(tickers) #3

model=LSTMStockModel(input_size, hidden_size, output_size)

loss_fn=nn.MSELoss()

optimizer=optim.Adam(model.parameters(), lr=0.001)

epochs=20

for epoch in range(epochs):
    epoch_loss=0
    for xb, yb in dataLoader:
        pred=model(xb)
        loss=loss_fn(pred,yb)
        optimizer.zero_grad() #기울기를 초기화 
        loss.backward()
        optimizer.step()
        epoch_loss +=loss.item()
print(f'Epoch : {epoch+1} / {epochs}, Loss : {epoch_loss/len(dataLoader):.4f}')

Epoch : 20 / 20, Loss : 0.0189


In [ ]:
# 추론 : 뉴데이터 입력해서 예측 
# 추론하려면 모드를 변경 
# 학습 모드 (model.train()), 추론 모드 (model.eval())
model.eval()
# 추론은 포워딩해야 한다. 백워드 (미분)하면 안된다
with torch.no_grad():
    preds=model(X_tensor).numpy() #StandardScaler 
    preds_original=scaler.inverse_transform(preds)
    
print(f'예측된 주가 : {preds_original[-1:]}') #최신 주가  


예측된 주가 : [[198.43996 148.84904 388.56   ]]


In [ ]:
# LSTM, GRU : 순환신경망 
# 문제점 : 장기기억이 잘 안된다, 병렬처리 비효율, 
# Transformer 계열의 신경망 활용 
# PatchTST 
# Informer 
# TimeNet : 푸리에변환, 시계열 데이터의 특성 추출 
